The aim of this preprocessing notebook is to run the preprocessing pipeline, whose goal is to create the following variables:
- an aggregated variable with betting odds: one column for an aggregated betting odd variable for home tema victory, one column for away team victory, one column for draw
- current season indicators, based on matched played during the current season
- absolute recent form indicators, based on the last matches played during the current season, regardless of the confronted teams
- absolute historical form indicators, based on the entire imported data (several seasons)
- relative recent form indicators, based on the last matches played against the same team
- strict relative recent form indicators, based on the last matches played against the same team in the same stadium (home away order kept)
- external factors which could impact a match issue

In [ ]:
import os
import sys
root_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(root_path)

import pandas as pd

from src.config import load_config
from src.preprocessing import Preprocessing
import src.utils

# config.yaml importation
config_file = 'config.yaml'
config_path = os.path.join(root_path, config_file)
config = load_config(config_path)

# Preprocessing pipeline for cleaned train and test dataframes

In [ ]:
cleaned_data_dir = os.path.join(root_path, config['cleaned_dir'])
cleaned_df_train_path = os.path.join(cleaned_data_dir, f"{config['cleaned_train_df_name']}.csv")
cleaned_df_test_path = os.path.join(cleaned_data_dir, f"{config['cleaned_test_df_name']}.csv")

cleaned_df_train = pd.read_csv(cleaned_df_train_path)
cleaned_df_test = pd.read_csv(cleaned_df_test_path)

In [ ]:
cleaned_df_train.head()

In [ ]:
cleaned_df_test.head()

In [ ]:
train_preproc = Preprocessing(cleaned_df_train, config)
test_preproc = Preprocessing(cleaned_df_test, config)

In [ ]:
preprocessed_df_train = train_preproc.run_preprocessing_pipeline()

In [ ]:
preprocessed_df_test = test_preproc.run_preprocessing_pipeline()

# Preprocessed dataframes export

In [ ]:
preprocessed_data_path = os.path.join(root_path, config['preprocessed_dir'])
df_train_path = os.path.join(preprocessed_data_path, f"{config['preprocessed_train_df_name']}.csv")
df_test_path = os.path.join(preprocessed_data_path, f"{config['preprocessed_test_df_name']}.csv")

preprocessed_df_train.to_csv(df_train_path, index=False)
preprocessed_df_test.to_csv(df_test_path, index=False)